<a href="https://colab.research.google.com/github/MicheleGiambelli/PlotTwisters-Project/blob/main/Transformer_funzionante_emma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np
import string
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

!pip install transformers datasets evaluate
!pip install transformers datasets evaluate seqeval

import os


from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification, pipeline
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from datasets import Dataset, DatasetDict
import evaluate
from sklearn.model_selection import train_test_split

import spacy
from collections import Counter, defaultdict
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import networkx as nx
from PIL import Image
import random
from itertools import combinations
from scipy.interpolate import make_interp_spline
from numpy.random import seed



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Pr

# **Caricamento dati + pulizia dataset**

In [3]:
# df1 = pd.read_csv('sample_data/file5.txt', header=None, delimiter='\t', names=['token', 'ner_tag'], skip_blank_lines=False)
# df1["token"] = df1["token"].fillna("end")
# df1["ner_tag"] = df1["ner_tag"].fillna("end")

# df2 = pd.read_csv('sample_data/file6.txt', header=None, delimiter='\t', names=['token', 'ner_tag'], skip_blank_lines=False)
# df2["token"] = df2["token"].fillna("end")
# df2["ner_tag"] = df2["ner_tag"].fillna("end")


def file_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Leggi tutte le righe del file
        for i, line in enumerate(lines):
            line = line.strip()  # Rimuove spazi e newline extra
            if not line:  # Linea vuota, segna come fine frase
                data.append(("end", "end"))
                continue

            # Controlla se la riga contiene '\t' per separare token e tag
            if '\t' in line:
                parts = line.split('\t')
                if len(parts) == 2:  # La riga è valida con 2 parti
                    token, ner_tag = parts
                    data.append((token, ner_tag))
                else:  # Riga malformata
                    print(f"Riga malformata alla linea {i + 1}: {line}")
            else:  # Riga senza '\t'
                print(f"Riga senza tabulazione alla linea {i + 1}: {line}")
                # data.append(("end", "end"))

    return pd.DataFrame(data, columns=['token', 'ner_tag'])

# Converte entrambi i file in DataFrame
df1 = file_to_dataframe('sample_data/file5.txt')
df2 = file_to_dataframe('sample_data/file5.txt')


#Assegna id di frase a file6
sentence_id = 0
for idx, row in df1.iterrows():
  df1.loc[idx, 'id'] = sentence_id
  if row['token'] == 'end' and row["ner_tag"]=="end":
    sentence_id += 1

for idx, row in df2.iterrows():
  df2.loc[idx, 'id'] = sentence_id
  if row['token'] == 'end' and row["ner_tag"]=="end":
    sentence_id += 1


df1 = df1[df1["ner_tag"] != "end"]
df2 = df2[df2["ner_tag"] != "end"]

combined_df = pd.concat([df1, df2], ignore_index=True)

# Convertire la colonna id da float a int
combined_df['id'] = combined_df['id'].astype(int)


combined_df = combined_df.dropna(subset=['token']).reset_index(drop=True) # con questo medodo possiamo rimuovere le righe del dataframe il cui valore è mancante

# Verifichiamo la presenza di righe vuote (serve più avanti per un problema al codice altrimenti)
invalid_tokens = combined_df[~combined_df['token'].apply(lambda x: isinstance(x, str))]

# Eliminare le righe con valori non stringa nella colonna 'token'
combined_df = combined_df[combined_df['token'].apply(lambda x: isinstance(x, str))].reset_index(drop=True)

Di seguito ho utilizzato la libreria open source *spaCy*, ideale per il Natural Language Processing (NLP). Ha supporto multilingue.

In [4]:
# Modello di lingua inglese di Spacy
nlp = spacy.load('en_core_web_sm')  # en_core_web_sm è un modello pre addestrato di spacy (small model).
stopwords = nlp.Defaults.stop_words
# Punteggiatura da rimuovere eslusa @
punctuation = set(string.punctuation) - {'@'}

# Funzione per eliminare stopwords e punteggiatura
def remove_stopwords_and_punctuation(df):
    df['token_cleaned'] = df['token'].apply(lambda x: x if x.lower() not in stopwords and x not in punctuation else '') # se è un carattere da rimuovere si crea riga vuota
    df = df[df['token_cleaned'] != '']  # Rimuove le righe con token vuoti
    return df.drop(columns=['token']).rename(columns={'token_cleaned': 'token'}) # si toglie la vecchia colonna "token" e si cambia il nome della nuova colonna "toen_cleaned" con "token"

# Applicare la funzione al dataset combinato
combined_df = remove_stopwords_and_punctuation(combined_df)

# Elimina le righe in cui il token è "rt"
combined_df = combined_df[combined_df['token'] != 'rt']  # "rt" sta per retweet

# Reset dell'indice per un DataFrame pulito (siccome prima abbiamo eliminato righe)
combined_df.reset_index(drop=True, inplace=True)

# Rimuovere eventuali NaN o valori non stringa nella colonna "token"
combined_df = combined_df.dropna(subset=['token']).reset_index(drop=True)
combined_df = combined_df[combined_df['token'].apply(lambda x: isinstance(x, str))].reset_index(drop=True) # manteniamo solo i valori stringa e togliamo il resto

# Dividere il dataset in train, validation e test
unique_ids = combined_df['id'].unique()
# seed(42) inutile perchè abbiamo già random_state=42 alla riga sotto
shuffled_ids = pd.Series(unique_ids).sample(frac=1, random_state=42).values
train_ids, temp_ids = train_test_split(shuffled_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=1/3, random_state=42)
train_df = combined_df[combined_df['id'].isin(train_ids)]
val_df = combined_df[combined_df['id'].isin(val_ids)]
test_df = combined_df[combined_df['id'].isin(test_ids)]

# Mappare le etichette NER a ID unici
all_df = pd.concat([train_df, val_df, test_df])                     # si poteva utilizzare direttamente combined_df
unique_tags = all_df['ner_tag'].unique().tolist()                   # .tolist() converte l'array NumPy restituito da .unique() in una lista Python.
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}          # dizionario in cui la chiave è il NER e il valore è un numero
id2tag = {idx: tag for tag, idx in tag2id.items()}                  # viceversa
train_df['ner_tag_id'] = train_df['ner_tag'].map(tag2id)            # ogni NER viene sostituito dal corrispondente id numerico presente nel dizionario tag2id
val_df['ner_tag_id'] = val_df['ner_tag'].map(tag2id)
test_df['ner_tag_id'] = test_df['ner_tag'].map(tag2id)




# ***Distilbert-base-cased***

Di seguito ho utilizzato un modello già attestrato chiamato ***distilbert-base-cased***.

Cos'è *distilbert-base-cased*:

*DistilBERT* è una versione più leggera e veloce di *BERT* (Bidirectional Encoder Representations from Transformers).                                      
Creato da *Hugging Face*, *DistilBERT* è stato ottenuto attraverso una tecnica chiamata distillazione del modello, che permette di comprimere un modello più grande (come BERT) in uno più piccolo mantenendo gran parte delle sue prestazioni.                                                                     
Il modello distilbert-base-cased è la versione di DistilBERT che distingue tra maiuscole e minuscole (case-sensitive).

Caratteristiche principali:

                                                                                     
1. Dimensioni ridotte: DistilBERT ha circa il 40% in meno di parametri rispetto a BERT base. Questo lo rende più leggero e più veloce da addestrare e inferire.


2. Prestazioni:Nonostante sia più piccolo, DistilBERT mantiene circa il 97% delle prestazioni di BERT su diversi benchmark NLP.


3. Vantaggi:

- Efficienza computazionale: Richiede meno memoria e risorse computazionali.
- Velocità: Più veloce sia durante l'addestramento che durante l'inferenza.
- Adatto per ambienti con risorse limitate: Ideale per applicazioni che necessitano di modelli leggeri.




Quando utilizzare distilbert-base-cased:

Quando hai limitazioni di risorse (RAM, GPU).
Se hai bisogno di addestrare o eseguire inferenze rapidamente.
Quando il leggero calo di prestazioni rispetto a BERT è accettabile per il tuo caso d'uso.


Di seguito metto alcune info del modello presenti al seguente link: https://huggingface.co/distilbert/distilbert-base-cased

- Distillation loss: the model was trained to return the same probabilities as the BERT base model.
- Masked language modeling (MLM): this is part of the original training loss of the BERT base model. When taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally mask the future tokens. It allows the model to learn a bidirectional representation of the sentence.
- Cosine embedding loss: the model was also trained to generate hidden states as close as possible as the BERT base model.

- Even if the training data used for this model could be characterized as fairly neutral, this model can have biased predictions. It also inherits some of the bias of its teacher model.
- DistilBERT pretrained on the same data as BERT, which is BookCorpus, a dataset consisting of 11,038 unpublished books and English Wikipedia (excluding lists, tables and headers).

In [ ]:
# Disabilitare wandb per evitare richieste di API key
os.environ["WANDB_DISABLED"] = "true"   # Disabilitare wandb (Weights and Biases): evita richieste di autenticazione a Weights and Biases, uno strumento di monitoraggio delle esperienze di machine learning.

# Per salvataggio modello (codice datato, più avanti lo miglioro, non lo tolgo altrimenti devo fare fine-tuning di nuovo)
save_directory = r'C:\Users\capel\OneDrive\Desktop\Data Visualization and Text Mining\Assignment'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)



####   **step 1: Mappare le etichette NER a ID unici** (fatto in precedenza)  #########


####   *step 2: Preparare i dati per Hugging Face**  ########

# Raggruppare i token e le etichette per frase utilizzando 'id' come identificatore siccome  modelli tipo BERT operano su frasi complete.
def group_data(df):
    return df.groupby('id').agg({'token': list, 'ner_tag_id': list}).reset_index()

train_dataset = group_data(train_df)
val_dataset = group_data(val_df)
test_dataset = group_data(test_df)


####   **step 3: Verificare l'allineamento tra token e etichette**   #####

def check_token_label_alignment(df_grouped):
    misaligned = df_grouped[
        df_grouped['token'].str.len() != df_grouped['ner_tag_id'].str.len()
    ]
    if not misaligned.empty:
        print("Frasi con disallineamento tra token e etichette:")
        print(misaligned)
    return misaligned.empty

assert check_token_label_alignment(train_dataset), "Disallineamento nel train set"
assert check_token_label_alignment(val_dataset), "Disallineamento nel validation set"
assert check_token_label_alignment(test_dataset), "Disallineamento nel test set"


####    *step 4: Creare i dataset per Hugging Face*    #####

train_dataset = Dataset.from_pandas(train_dataset)  # siccome hugging face utilizza specifici formati utilizziamo "Dataset" per convertire un DataFrame Pandas in un formato compatibile
val_dataset = Dataset.from_pandas(val_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

datasets = DatasetDict({   # raggruppiamo tutto insieme per facilita l'uso delle API di hugging face.
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})


####  **step 5: Caricare il tokenizer e il modello pre-addestrato**  #######

model_name = "distilbert-base-cased"                   # Modello leggero e case-sensitive
tokenizer = AutoTokenizer.from_pretrained(model_name)  # AutoModelForTokenClassification è il modello pre-addestrato, configurato per il task di token classification (NER).
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(tag2id)                 # specificare il numero di token unici con num_labels=len(tag2id)
)


####   **step 6: Tokenizzazione e allineamento delle etichette**   ######

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['token'],
        is_split_into_words=True,  # token già divisi in parole (e non frasi)
        truncation=True,           # troncare token che superano la lunghezza massima (non ci dovrebbero essere token da 65 in ogni caso)
        padding='max_length',      # ricordarsi: il padding è una tecnica utilizzata per gestire input di lunghezza differente. aggiunge token speciali ([PAD]) alle sequenze più corte per uniformarne la lunghezza.
        max_length=65              # Lunghezza massima pari a 65
    )

    labels = []
    for i, label in enumerate(examples['ner_tag_id']):
        word_ids = tokenized_inputs.word_ids(batch_index=i) # restituisce None per i token speciali come [PAD], [CLS] e [SEP]
                                                            # ogni elemento della lista rappresenta l'indice della parola originale a cui il token appartiene oppure None
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:                   # si controlla token speciali tipo [CLS], [SEP] e nel caso gli si assegna un etichetta particolare per ignorarli
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Token principali
            else:                                  # Gestisce i sotto-token, cioè i token che appartengono alla stessa parola (per sicurezza)
                label_ids.append(-100)             # Sotto-token
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Applicare la tokenizzazione ai dataset
print("Tokenizzazione in corso...")
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,                    # è la funzione definita prima
    batched=True,                                 # più frasi in contemporanea
    remove_columns=['id', 'token', 'ner_tag_id']
)




#### **step 7: Preparare per l'addestramento**  ######

data_collator = DataCollatorForTokenClassification(tokenizer)  # Data Collator: gestisce il padding dinamico durante la creazione dei batch, in modo che ogni batch sia della stessa lunghezza del token più lungo presente. Questo aumenta l'efficienza e la gestione della memoria.

# Definire la funzione per calcolare le metriche
def compute_metrics(p):
    predictions, labels = p                                    # Rappresenta le predizioni del modello e le etichette vere
    predictions = np.argmax(predictions, axis=2)               # Converte le predizioni probabilistiche del modello in etichette finali

    true_predictions = [
        [id2tag[pred] for (pred, label_id) in zip(prediction, label) if label_id != -100]    # escludiamo i lable -100 ovvero caratteri speciali
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[label_id] for (pred, label_id) in zip(prediction, label) if label_id != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Sequeval per il calcolo delle metriche
    seqeval = evaluate.load("seqeval")                       # SEQUEVAL:  è una libreria Python progettata per calcolare metriche di valutazione per il NER.
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results.get("overall_precision", 0.0),  #  Percentuale di entità rilevate correttamente rispetto a tutte quelle predette.
        "recall": results.get("overall_recall", 0.0),        # Percentuale di entità rilevate correttamente rispetto a tutte quelle effettivamente presenti.
        "f1": results.get("overall_f1", 0.0),                # Media armonica tra precisione e richiamo
        "accuracy": results.get("overall_accuracy", 0.0),    # Percentuale di token classificati correttamente (nel validation)

    }






###    **step 8: Impostare i parametri di addestramento* ######à

training_args = TrainingArguments(
    output_dir=save_directory,        # Directory in cui vorrei salvare il modello (definita in precedenza)
    num_train_epochs=3,               # il modello farà 3 volte il giro del trainig set
    per_device_train_batch_size=32,   # numero di campioni processati in parallelo durante l'addestramento e la valutazione.
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",      # valutare il modello al termine di ogni epoca
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=50,                # Specifica dove salvare i log e ogni quanti step loggare le informazioni di addestramento (50 dovrebbe essere la prassi)
    load_best_model_at_end=True,     # Dopo l'addestramento, carica automaticamente il modello con le migliori prestazioni sulla validation set
    report_to="none",
    fp16=torch.cuda.is_available(),  # Utilizza calcoli a precisione mista (FP16) se una GPU con supporto CUDA è disponibile, per migliorare la velocità e ridurre il consumo di memoria.
)





#####  **step 9: Creare il trainer**  ######

trainer = Trainer(                                    # "Trainer" in hugging face che include una variante della Cross Entropy Loss la quale ignora automaticamente i token con etichetta -100 (quindi ad esempio i padding)
    model=model,                                      # modello pre addestrato
    args=training_args,
    train_dataset=tokenized_datasets['train'],        # TRAINING SET UTILIZZATO QUI, ricordarsi che contiene sequenze di stringe e -100
    eval_dataset=tokenized_datasets['validation'],    # VALIDATION SET UTILIZZATO QUI
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



###  step 10: Avviare l'addestramento  ###

print("Inizio dell'addestramento...")
trainer.train()





Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizzazione dei dataset in corso...


Map:   0%|          | 0/2993 [00:00<?, ? examples/s]

Map:   0%|          | 0/855 [00:00<?, ? examples/s]

Map:   0%|          | 0/428 [00:00<?, ? examples/s]

Inizio dell'addestramento...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.672800,0.207359,0.802809,0.765399,0.783658,0.930994
2,0.157300,0.136910,0.867406,0.879486,0.873404,0.958503
3,0.078600,0.121626,0.889655,0.898232,0.893923,0.965031


TrainOutput(global_step=282, training_loss=0.23686940365649284, metrics={'train_runtime': 2934.102, 'train_samples_per_second': 3.06, 'train_steps_per_second': 0.096, 'total_flos': 146655030462336.0, 'train_loss': 0.23686940365649284, 'epoch': 3.0})

# **Metriche per Epoca durante l'Addestramento:**

Nella tabella, possiamo vedere il progresso del modello in termini di perdita ("loss") e metriche di valutazione durante le tre epoche di addestramento.

***Training Loss e Validation Loss:***

La Training Loss e la Validation Loss diminuiscono progressivamente per ciascuna epoca. Questo indica che il modello sta migliorando la sua capacità di apprendere i pattern dai dati, riducendo sia l'errore sui dati di training che l'errore sui dati di validazione. Una diminuzione consistente della Validation Loss suggerisce che il modello non sta overfittando, ma sta generalizzando bene su dati non visti.



***Precision, Recall, F1, e Accuracy:***


*Precision*: Rappresenta la percentuale di previsioni corrette tra tutte le previsioni fatte. Vediamo che la precision migliora da 0.802 nella prima epoca a 0.889 nella terza epoca.

*Recall*: Indica la percentuale di entità corrette che il modello è stato capace di individuare. Anche il recall aumenta da 0.765 a 0.898, indicando che il modello è stato in grado di riconoscere più entità correttamente man mano che l'addestramento progrediva.

*F1 Score*: La media armonica di precision e recall, che rappresenta un buon compromesso tra le due metriche.

*Accuracy*: La Accuracy complessiva sale mostrando un incremento costante nell'accuratezza del modello.

In generale, possiamo notare che i valori di tutte le metriche migliorano durante le tre epoche, indicando che il modello è stato in grado di apprendere e migliorare efficacemente.

In [ ]:
#### step 11: Valutare il modello sui vari set  ####

# Valutare il modello sul set di validazione
print("Valutazione del modello sul validation set...")
validation_metrics = trainer.evaluate(tokenized_datasets['validation'])
print("Metriche sul validation set:")
print(validation_metrics)

# Valutare il modello sul set di test
print("Valutazione del modello sul test set...")
test_metrics = trainer.evaluate(tokenized_datasets['test'])
print("Metriche sul test set:")
print(test_metrics)


# Raccolta metriche
data = {
    "Set": ["Validation", "Test"],
    "Loss": [validation_metrics['eval_loss'], test_metrics['eval_loss']],
    "Precision": [validation_metrics.get('eval_precision', 0.0), test_metrics.get('eval_precision', 0.0)],  #0.0 serve per evitare errori nel caso in cui la metrica non sia disponibile
    "Recall": [validation_metrics.get('eval_recall', 0.0), test_metrics.get('eval_recall', 0.0)],
    "F1": [validation_metrics.get('eval_f1', 0.0), test_metrics.get('eval_f1', 0.0)],
    "Accuracy": [validation_metrics.get('eval_accuracy', 0.0), test_metrics.get('eval_accuracy', 0.0)]
}

# Creazione di un DataFrame Pandas per creare tabella leggibile
metrics_df = pd.DataFrame(data)
print("\nTabella riassuntiva delle metriche per Validation e Test:")
print(metrics_df)





#### **step 12: Salvare il modello addestrato**  ####   questo codice è obsoleto

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)


Valutazione del modello sul set di validazione in corso...


Metriche del set di validazione:
{'eval_loss': 0.12162584811449051, 'eval_precision': 0.8896551724137931, 'eval_recall': 0.8982324584895555, 'eval_f1': 0.8939232409381662, 'eval_accuracy': 0.9650308893810468, 'eval_runtime': 95.1234, 'eval_samples_per_second': 8.988, 'eval_steps_per_second': 0.284, 'epoch': 3.0}
Valutazione del modello sul set di test in corso...
Metriche del set di test:
{'eval_loss': 0.12932206690311432, 'eval_precision': 0.8754134509371555, 'eval_recall': 0.8861607142857143, 'eval_f1': 0.8807542983915697, 'eval_accuracy': 0.962435837610826, 'eval_runtime': 39.7975, 'eval_samples_per_second': 10.754, 'eval_steps_per_second': 0.352, 'epoch': 3.0}

Tabella riassuntiva delle metriche per Validation e Test:
          Set      Loss  Precision    Recall        F1  Accuracy
0  Validation  0.121626   0.889655  0.898232  0.893923  0.965031
1        Test  0.129322   0.875413  0.886161  0.880754  0.962436


# **Tabella Riassuntiva delle Metriche per Validation e Test**

Nella tabella, vengono rappresentate delle metriche calcolate sui set di validazione e di test, dopo il completamento delle tre epoche di addestramento.

***Validation Set:***

I valori suggeriscono che il modello è stato in grado di generalizzare piuttosto bene, con una buona precision e un buon recall.


***Test Set:***
Valori molto simili al validation.


Interpretazione Complessiva:
- In generale, osserviamo che il modello mostra buoni risultati sia sui dati di validazione che di test. L'accuratezza e l'F1 score relativamente alti suggeriscono che il modello ha appreso a riconoscere le entità nominate in maniera affidabile e che il rischio di overfitting è basso.
- Il Recall relativamente alto rispetto alla Precision suggerisce che il modello è piuttosto "inclusivo", cercando di identificare la maggior parte delle entità, anche se questo significa che occasionalmente potrebbe fare delle previsioni meno precise (ovvero, predire un'entità dove non c'è).
- La coerenza tra le metriche di validazione e di test è un buon segnale, indicando che il modello ha una capacità di generalizzazione sufficiente su dati mai visti prima, senza aver appreso pattern troppo specifici del training set.

In [ ]:
####### SALVARE IL MODELLO  sul drive  #######

from google.colab import drive
drive.mount('/content/drive')

save_directory = '/content/drive/MyDrive/Assignment'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modello e tokenizer salvati correttamente nella directory: {save_directory}")

Mounted at /content/drive
Modello e tokenizer salvati correttamente nella directory: /content/drive/MyDrive/Assignment


# **Testare il modello di predizione delle etichette NER**

Di seguito lo testo sia sul modello non fine-tunato sia su quello fine-tunato.

In [ ]:
####  TESTIAMO IL MODELLO  (ma sul modello di hugging face e non sul modello fine-tunato, dopo lo farò pure su quello), è una prova   ########

def predict_ner(sentence):
    # Tokenizzare la frase (ovvero trasforma ogni parola in un ID numerico.)
    inputs = tokenizer.encode_plus(
        sentence,
        return_tensors='pt',             # Restituisce l'output come tensori PyTorch
        is_split_into_words=False,       # input è una frase cimpleta
        truncation=True,
        max_length=65                    # lunghezza max usata durante l'addestramento
    )
    input_ids = inputs['input_ids']


    model.eval()                         # modello in modalità valutazione per evitare modifiche dei pesi

    # Ottenere le predizioni
    with torch.no_grad():                # Disabilita il calcolo del gradiente
        outputs = model(input_ids)       # si ottengono i logits che sono le probabilità non normalizzate per ogni token e per ogni classe NER
                                         # qua utilizzo il modello addestrato prima
    predictions = torch.argmax(outputs.logits, dim=2)[0].numpy()    # Per ogni token si seleziona l'indice della classe con la probabilità più alta

    # Convertire input_ids in token
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Ricostruire le parole dai sub-token e assegnare le etichette
    words = []
    labels = []
    for idx, (token, pred) in enumerate(zip(tokens, predictions)):
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue  # Ignora token speciali
        if token.startswith('##'):
            words[-1] += token[2:]
        else:
            words.append(token)
            labels.append(id2tag[pred])

    # Stampare le parole e le relative etichette
    print("\nPredizione:")
    print("Parola\tEtichetta")
    for word, label in zip(words, labels):
        print(f"{word}\t{label}")

In [ ]:
###  Esempi di predizione   ###
print("\nEsempio di predizione:")
test_sentence = "let's see if the predictions of @Plottwisters are correct and look if Emma has done a great job here in milan."
predict_ner(test_sentence)


Esempio di predizione:
Parola	Etichetta
let	O
'	O
s	O
see	O
if	O
the	O
predictions	O
of	O
@	B-PER
Plottwisters	B-PER
are	O
correct	O
and	O
look	O
if	O
Emma	B-PER
has	O
done	O
a	O
great	O
job	O
here	O
in	O
milan	B-LOC
.	O


In [8]:
test_sentence = "Hi Emma in Milan, !!! America, Obama, Trump and NATO @Plottwister is predicting pretty well."
predict_ner(test_sentence)


Predizione:
Parola	Etichetta
Hi	O
Emma	B-PER
in	O
Milan	B-LOC
,	O
!	O
!	O
!	O
America	B-PER
,	O
Obama	B-PER
,	O
Trump	B-PER
and	O
NATO	B-PER
@	B-PER
Plottwister	B-PER
is	O
predicting	O
pretty	O
well	O
.	O


In [9]:
####### TEST DI PREDIZIONE SUL MODELLO FINE TUNATO ###############


# Specificare la directory dove sono salvati i file ("./sample_data" è la mia ad esmpio)
model_directory = "./sample_data"

# Caricare il modello fine-tunato e il tokenizer
model = AutoModelForTokenClassification.from_pretrained(model_directory)  # Carica il modello fine-tunato
                                                                          # config: Per configurare il modello.
                                                                          # model.safetensors: Per caricare i pesi addestrati.
                                                                          # tokenizer_config, special_tokens_map, vocab: Per configurare il tokenizer e mappare i token ai rispettivi ID
tokenizer = AutoTokenizer.from_pretrained(model_directory)                # Carica il tokenizer

# Funzione di predizione
def predict_ner(sentence):
    # Tokenizzare la frase
    inputs = tokenizer.encode_plus(
        sentence,
        return_tensors='pt',             # Restituisce l'output come tensori PyTorch
        is_split_into_words=False,       # Input è una frase completa
        truncation=True,
        max_length=65                    # Lunghezza max usata durante l'addestramento
    )
    input_ids = inputs['input_ids']

    model.eval()                         # Modello in modalità valutazione per evitare modifiche ai pesi

    # Ottenere le predizioni
    with torch.no_grad():                # Disabilita il calcolo del gradiente
        outputs = model(input_ids)       # Si ottengono i logits
    predictions = torch.argmax(outputs.logits, dim=2)[0].numpy()    # Per ogni token si seleziona l'indice della classe con la probabilità più alta

    # Convertire input_ids in token
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Ricostruire le parole dai sub-token e assegnare le etichette
    words = []
    labels = []
    for idx, (token, pred) in enumerate(zip(tokens, predictions)):
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue
        if token.startswith('##'):
            words[-1] += token[2:]
        else:
            words.append(token)
            labels.append(id2tag[pred])

    # Stampare le parole e le relative etichette
    print("\nPredizione:")
    print("Parola\tEtichetta")
    for word, label in zip(words, labels):
        print(f"{word}\t{label}")


print("\nEsempio di predizione:")
test_sentence = "let's see if the predictions of @Plottwisters are correct and look if Emma has done a great job here in milan."
predict_ner(test_sentence)
test_sentence = "Hi Emma in Milan, !!! America, Obama, Trump and NATO @Plottwister is predicting pretty well."
predict_ner(test_sentence)


Esempio di predizione:

Predizione:
Parola	Etichetta
let	O
'	O
s	O
see	O
if	O
the	O
predictions	O
of	O
@	B-PER
Plottwisters	B-PER
are	O
correct	O
and	O
look	O
if	O
Emma	B-PER
has	O
done	O
a	O
great	O
job	O
here	O
in	O
milan	B-LOC
.	O

Predizione:
Parola	Etichetta
Hi	O
Emma	B-PER
in	O
Milan	B-LOC
,	O
!	O
!	O
!	O
America	B-PER
,	O
Obama	B-PER
,	O
Trump	B-PER
and	O
NATO	B-PER
@	B-PER
Plottwister	B-PER
is	O
predicting	O
pretty	O
well	O
.	O
